In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

Setup the device


In [2]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print("device: ", device)
torch.device(device)

device:  mps


device(type='mps')

CFar data

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


Build network

In [ ]:
import torch.nn as nn

class NNModel (nn.Module):
    def __init__(self, input_size):
        super(NNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 12, 3, padding='same')
        self.batch1 = nn.BatchNorm2d(12)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(12, 32, 3, padding='same')
        self.batch2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3, padding='same')
        self.batch3 = nn.BatchNorm2d(64)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.10)


    def forward(self, x):
        x = self.conv1(x)
        x = self.batch1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv3(x)
        x = self.batch3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        #x = self.dropout(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        return x

In [5]:
from torchinfo import summary

model = NNModel(10)
summary(model, input_size=(64, 3, 32, 32), device='cpu', col_names=['input_size', 'output_size',
                                                                           'num_params'])



/Users/manuelrodriguez/miniconda3/envs/pytorch2/lib/python3.8/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/Users/manuelrodriguez/miniconda3/envs/pytorch2/lib/python3.8/site-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
NNModel                                  [64, 3, 32, 32]           [64, 10]                  --
├─Conv2d: 1-1                            [64, 3, 32, 32]           [64, 12, 32, 32]          336
├─BatchNorm2d: 1-2                       [64, 12, 32, 32]          [64, 12, 32, 32]          24
├─ReLU: 1-3                              [64, 12, 32, 32]          [64, 12, 32, 32]          --
├─MaxPool2d: 1-4                         [64, 12, 32, 32]          [64, 12, 16, 16]          --
├─Conv2d: 1-5                            [64, 12, 16, 16]          [64, 32, 16, 16]          3,488
├─BatchNorm2d: 1-6                       [64, 32, 16, 16]          [64, 32, 16, 16]          64
├─ReLU: 1-7                              [64, 32, 16, 16]          [64, 32, 16, 16]          --
├─MaxPool2d: 1-8                         [64, 32, 16, 16]          [64, 32, 8, 8]            --
├─Conv2d: 1-9                  

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
lost_list = []

In [7]:
epochs = 10
for epoch in range(epochs):  # loop over the dataset multiple times
    print("Epoch: ", epoch)
    print("--------------") 

    running_loss = 0.0
    size = len(trainloader.dataset)
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        
        if i % 100 == 0:
            loss, current = loss.item(), i * len(inputs)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
        # print statistics
        #running_loss += loss.item()
        #if i % 2000 == 1999:    # print every 2000 mini-batches
        #    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
        #    running_loss = 0.0
        
    size = len(testloader.dataset)
    num_batches = len(testloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in testloader:
            #X = X.to(device)
            #y = y.to(device)
            pred = model(X)
            test_loss += criterion(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


print('Finished Training')

Epoch:  0
--------------
loss: 2.305736  [    0/50000]
loss: 2.097169  [ 6400/50000]
loss: 2.220761  [12800/50000]
loss: 2.165665  [19200/50000]
loss: 2.013907  [25600/50000]
loss: 1.776513  [32000/50000]
loss: 1.959481  [38400/50000]
loss: 1.861729  [44800/50000]
Test Error: 
 Accuracy: 34.2%, Avg loss: 1.864391 

Epoch:  1
--------------
loss: 1.727889  [    0/50000]
loss: 1.784922  [ 6400/50000]
loss: 1.724765  [12800/50000]
loss: 1.922395  [19200/50000]
loss: 1.913162  [25600/50000]
loss: 1.888348  [32000/50000]
loss: 1.912720  [38400/50000]
loss: 1.892442  [44800/50000]
Test Error: 
 Accuracy: 36.6%, Avg loss: 1.763880 

Epoch:  2
--------------
loss: 1.716307  [    0/50000]
loss: 1.578296  [ 6400/50000]


KeyboardInterrupt: 